# Named Entity Extractor with Spacy
### By **Néstor Suat** in 2021

**Descripción:** Con el modelo Spacy entrenado previamente se extrae las entidades **loc** y **time**

**Input:**
* TSV con tweets

**Output:**
* TSV agregando columna clean y entities

***

### Importando librerías e Inicializando variables

In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../../../')

from classes.tweet2accident.ner_preprocessing import NerPreprocessing
from classes.tweet2accident.ner_extractor import NerExtractor

In [2]:
## Variables para importar modelos y demás
dir_ = "../../../data/v1/NER/"
dir_dataset = "../../../data/database/output_ml/M1/"

#file = 'ner_dataset.tsv' # Dataset
file = 'accident_1_server_bogota.tsv' # Dataset

spacy_model = dir_+"spacy_model_complete/" #Spacy model entrenado previamente
corpus_segmentation = dir_+'spanish_count_1w_small_v2_twitter.txt' # Corpus para entrenar el wordsemgentation

In [3]:
## Importando Dataset
dataset = pd.read_csv(dir_dataset+file, delimiter = "\t", quoting = 3)
del dataset['Unnamed: 0']
print(dataset.shape)
dataset.head(5)

(5832, 25)


,_id,api,coordinates,created_at,created_at_str,favorite_count,geo,id_tweet,place_coordinates,place_country,...,user_followers_count,user_friends_count,user_id,user_listed_count,user_location,user_name,user_profile_image_url,user_statuses_count,user_verified,label
0,5bb4a18069d02d6942f0b4e0,Stream,{},2018-10-03 06:01:20,2018-10-03 11:01:20,0,{},1047441463322316800,"{'type': 'Polygon', 'coordinates': [[[{'$numbe...",Colombia,...,21261,1533,189192159,226.0,Bogotá - Colombia,EsperanzaRicoL,http://pbs.twimg.com/profile_images/9887424484...,88093,False,1
1,5bb4b8b969d02d78e814fbbc,Stream,{},2018-10-03 07:40:24,2018-10-03 12:40:24,0,{},1047466397146796034,"{'type': 'Polygon', 'coordinates': [[[{'$numbe...",Colombia,...,709,4996,4798808621,1.0,NaN,camilogallardoa,http://pbs.twimg.com/profile_images/6874214127...,13469,False,1
2,5bb4e90069d02d297e7df854,Stream,{},2018-10-03 11:06:24,2018-10-03 16:06:24,0,{},1047518237301325825,"{'type': 'Polygon', 'coordinates': [[[{'$numbe...",Colombia,...,3417,142,796390153191387137,18.0,"Meta, Colombia",PoliciaMeta,http://pbs.twimg.com/profile_images/9847893547...,13900,True,1
3,5bb4f1c669d02d303fad6c8f,Stream,{},2018-10-03 11:43:49,2018-10-03 16:43:49,0,{},1047527654830891008,"{'type': 'Polygon', 'coordinates': [[[{'$numbe...",Colombia,...,621,455,602432456,2.0,Combita,JavierAraqueEla,http://pbs.twimg.com/profile_images/8451021853...,839,False,1
4,5bb5541e69d02d0680213e47,Stream,"{'type': 'Point', 'coordinates': [{'$numberDou...",2018-10-03 18:43:26,2018-10-03 23:43:26,0,"{'type': 'Point', 'coordinates': [{'$numberDou...",1047633252834594816,"{'type': 'Polygon', 'coordinates': [[[{'$numbe...",Colombia,...,2,88,933487951023169536,0.0,"Bogotá, D.C., Colombia",JPRUIZ1995,http://pbs.twimg.com/profile_images/9572315669...,169,False,1


In [4]:
dataset.shape

(5832, 25)

### Limpieza y normalización para NER

In [5]:
ner_preprocessing = NerPreprocessing(spacy_model=spacy_model, corpus_segmentation=corpus_segmentation,njobs=4)
txt = ner_preprocessing.transform(dataset['text'])
dataset['clean'] = txt

In [6]:
dataset[['text','clean']].head(5)

,text,clean
0,Desmanes en Bogotá dejaron anoche dos personas...,Desmanes en Bogotá dejaron anoche dos personas...
1,Rt grave accidente en cumaral un camión choca ...,Rt grave accidente en cumaral un camión choca ...
2,Art. 141 Derecho de vía de peatones y ciclista...,Art. 141 Derecho de vía de peatones y ciclista...
3,"Instituto de Tránsito de Boyacá, Secretaria de...","Instituto de Tránsito de Boyacá, Secretaria de..."
4,Acaba de publicar una foto en Clinica Colsubsi...,Acaba de publicar una foto en Clinica Colsubsi...


### Predicción Etiquetar NER
Etiquetas **'loc'** y **'time'**

In [7]:
dataset.iloc[1]['clean']

'Rt grave accidente en cumaral un camión choca de frente contra una aerovans 2 personas fallecidas y seis heridos hasta el momento la vía esta cerrada en estos momentoscierre total de via Cumaral - Restrepo por accidente de transito, Km 25 500 entrada a cumaral'

In [8]:
ner_extractor = NerExtractor(spacy_model=spacy_model, njobs=4)
txt = ner_extractor.transform(dataset['clean'])
dataset['entities'] = txt 

In [9]:
dataset[['text','entities']].head(5)

,text,entities
0,Desmanes en Bogotá dejaron anoche dos personas...,"[(santafe, loc)]"
1,Rt grave accidente en cumaral un camión choca ...,"[(Km 25 500, loc)]"
2,Art. 141 Derecho de vía de peatones y ciclista...,"[(Art 141, loc)]"
3,"Instituto de Tránsito de Boyacá, Secretaria de...","[(Instituto, loc), (Boyacá, loc), (Buenaventur..."
4,Acaba de publicar una foto en Clinica Colsubsi...,"[(Clinica Colsubsidio Calle 100, loc)]"


In [10]:
dataset.shape

(5832, 27)

In [11]:
dataset.to_csv(dir_dataset+"NER_extractor/entities_"+file,sep='\t',index=False)

---
### Pruebas

In [9]:
i = 46
print(dataset.iloc[i]['text'])
print(dataset.iloc[i]['entities'])
print(type(dataset.iloc[i]['entities']))
print(dataset.iloc[i]['entities'][0])
print(type(dataset.iloc[i]['entities'][0]))

#AEstaHora (6:00 P.M.) se registra una fuerte congestión vehicular sobre la Cll 26 en sentido Oriente-Occidente hasta el @BOG_ELDORADO por dos choques menores. Te recomendamos salir con tiempo de anticipación para cumplir con tu plan de vuelo. #Comparte #QueNoTeDejeElAvión https://t.co/0jb7hPHlWc
[('esta hora', 'time'), ('6:00 P.M.', 'time'), ('Cll 26', 'loc')]
<class 'list'>
('esta hora', 'time')
<class 'tuple'>


In [10]:
ent = [ t for (t,l) in dataset.iloc[i]['entities']  if l == 'loc' ]
ent

['Cll 26']

In [11]:
ent = [ t for (t,l) in dataset.iloc[i]['entities']  if l == 'time' ]
ent

['esta hora', '6:00 P.M.']

### Exportar tsv

In [12]:
dataset.to_csv(dir_+"ner_dataset_entities.tsv",sep='\t')